In [ ]:
#!/usr/bin/env python3
#  -*- coding: utf-8 -*-
"""
A bot must be created and pointed to this server in the My Apps section of
https://developer.webex.com.  The bot's Access Token should be added as a
"WEBEX_TEAMS_ACCESS_TOKEN" environment variable on the web server hosting this
script.
This script must expose a public IP address in order to receive notifications
about Webex events.  ngrok (https://ngrok.com/) can be used to tunnel traffic
back to your server if your machine sits behind a firewall.
The following environment variables are needed for this to run
* WEBEX_TEAMS_ACCESS_TOKEN -- Access token for a Webex bot
* WEBHOOK_URL -- URL for Webex Webhooks (ie: https://2fXX9c.ngrok.io)
* PORT - Port for Webhook URL (ie: the port param passed to ngrok)
This sample script leverages the Flask web service micro-framework
(see http://flask.pocoo.org/).  By default the web server will be reachable at
port 5000 you can change this default if desired (see `flask_app.run(...)`).
In our app we read the port from the PORT environment variable.
Upon startup this app create webhooks so that our bot is notified when users
send it messages or interact with any cards that have been posted. 
This script should support Python versions 3.6+ only.
Copyright (c) 2016-2020 Cisco and/or its affiliates.
Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:
The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.
THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.
"""

import json
import os
import sys
from urllib.parse import urljoin
import requests
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
import time
import statistics
import matplotlib.pyplot as plt
from datetime import date




# Script metadata
__author__ = "Healkan Cheung"
__author_email__ = "heacheunr@cisco.com"
__copyright__ = "Copyright (c) 2016-2020 Cisco and/or its affiliates."
__license__ = "MIT"


# Constants

output_file = 'LDOS-out.txt'
input_file = 'LDOS-input-test.txt'





# Get Cisco API Auth token
def getnewtoken():
    
    # Get API authn client id and secret from file api_cred
    from api_cred import client_id
    from api_cred import client_secret

    # Remember the current time.  This is used to determine if token needs to be refreshed.
    now = time.time()

    # URL for REST call to generate new token
    url = "https://cloudsso.cisco.com/as/token.oauth2?client_id=" + client_id + "&grant_type=client_credentials&client_secret=" + client_secret

    payload = {}
    headers = {
      'Content-Type': 'application/x-www-form-urlencoded',
      'Cookie': 'PF=79XZT8hLaZ8I29ms2LsKo7'
    }

    response = requests.request("POST", url, headers=headers, data = payload)
  
    tresponse = json.loads(response.text)
    token = tresponse["access_token"]

    # Saving token to be used by Cisco APIs
    token = 'Bearer ' + token
    
    return token,now

# Check if API token needs to be refreshed
def refreshtoken(token,now):

    # Get current time and check if the time since the auth token was created. 
    later = time.time()
    difference = int(later - now)

    #  If time since token was created is greater than 3500 seconds, then get a new token. 
    #  3599 seconds is the documented expiration period, but we use 3500 sec to be safe.
    if difference >= 3500:
        print('Refresh new token')
        token,now = getnewtoken()
    return token,now

# Function to call CCW-R API
def ccw_r_api(serial,token):
    url = "https://api.cisco.com/ccw/renewals/api/v1.0/search/lines"

    # Payload contains a list of up to 20 serial numbers.  
    payload = '{\n    \"serialNumbers\": [\n        ' + serial + '\n    ],\n    \"offset\": 0,\n    \"limit\": 1000,\n    \"configurations\": true\n}'
    headers = {
      'Content-Type': 'application/json',
      'Authorization': token,
      'Accept-Language': 'en_US',
      'Request-id': '78909092'
    }

    response = requests.request("POST", url, headers=headers, data = payload)

    # saving CCW-R API output in a JSON dictionary
    ccw_r_response = json.loads(response.text)

    return ccw_r_response
    


# Function to call EOX API
def eox_api(pid,token):

    # Retrieving EOX info for a serial number
    url = "https://api.cisco.com/supporttools/eox/rest/5/EOXByProductID/1/" + pid + "?responseencoding=json"

    payload = {}
    headers = {
      'Accept': 'application/json',
      'Authorization': token
    }

    # Code to retry API call if timing out
    with requests.Session() as s:
        retries = Retry(
            total=10,
            backoff_factor=0.2,
            status_forcelist=[500, 502, 503, 504])

    s.mount('http://', HTTPAdapter(max_retries=retries))
    s.mount('https://', HTTPAdapter(max_retries=retries))
    response = s.get(url,headers=headers, data = payload)
    
    # saving EOX API output in a JSON dictionary      
    eox_api_response = json.loads(response.text)
        
    return eox_api_response


def show_ldos(inventory,first_year,last_year):

    outfile = open(output_file,'w')   
    replacement = {}
    for year in range(first_year, last_year+1):
        replacement[str(year)]={}
        years_shipped = []
        print('\033[1m',"\n\nLDOS for Year ", str(year),'\033[0m')
        outfile.writelines(["\n LDOS for Year ", str(year),"\n"])
        num_LDOS = 0
        for dev in inventory:
            if str(year) in dev["LDOSDate"]:
                num_LDOS += 1
                print("Serial: ",dev["Serial"]," Site: ", dev["Site"], " Product: ", dev["Product"]," LDOS Date: ",dev["LDOSDate"]," Replacement: ",dev["Replacement"], " Ship Date: ", dev["ShipDate"])
                outfile.writelines(["Serial: ",dev["Serial"]," Site: ", dev["Site"], " Product: ", dev["Product"]," LDOS Date: ",dev["LDOSDate"]," Replacement: ",dev["Replacement"], " Ship Date: ", dev["ShipDate"],"\n"])

                if dev["Replacement"] in replacement[str(year)].keys():
                    replacement[str(year)][dev["Replacement"]] += 1
                else:
                    replacement[str(year)][dev["Replacement"]] = 1
                if dev["ShipDate"] != "":
                    yr = int(dev["ShipDate"][0:4])
                    mo= int(dev["ShipDate"][5:7])
                    day= int(dev["ShipDate"][8:10])
                    f_date = date(yr, mo, day)
                    l_date = date(2020, 10, 19)
                    delta = l_date - f_date
                    years_shipped.append(delta.days/365)
        print('\033[92m',"Replacement ", str(year),": ", str(replacement[str(year)]),'\033[0m')
        outfile.writelines(["\nReplacement ", str(year),": ", str(replacement[str(year)]),"\n"])
        
        if years_shipped:
            print("Median age of LDOS gear = ", str(round(statistics.median(years_shipped),2)), " years.  ", "Oldest gear is: ", str(round(max(years_shipped),2)), " years old." )

    # Showing devices with no LDOS
    years_shipped = []
    print("\n\nNo LDOS")
    for dev in inventory:
        if dev["LDOSDate"] == "":

            print("Serial: ",dev["Serial"]," Site: ", dev["Site"], " Product: ", dev["Product"]," LDOS Date: ",dev["LDOSDate"]," Replacement: ",dev["Replacement"], " Ship Date: ", dev["ShipDate"])
            outfile.writelines(["Serial: ",dev["Serial"]," Site: ", dev["Site"], " Product: ", dev["Product"]," LDOS Date: ",dev["LDOSDate"]," Replacement: ",dev["Replacement"], " Ship Date: ", dev["ShipDate"],"\n"])

            if dev["ShipDate"] != "":
                yr = int(dev["ShipDate"][0:4])
                mo= int(dev["ShipDate"][5:7])
                day= int(dev["ShipDate"][8:10])
                f_date = date(yr, mo, day)
                l_date = date(2020, 10, 19)
                delta = l_date - f_date
                years_shipped.append(delta.days/365)
    if years_shipped:
        print("Median age of LDOS gear = ", str(round(statistics.median(years_shipped),2)), " years.  ", "Oldest gear is: ", str(round(max(years_shipped),2)), " years old.","\n"  )
        outfile.writelines(["\n Median age of LDOS gear = ", str(round(statistics.median(years_shipped),2)), " years.  ", "Oldest gear is: ", str(round(max(years_shipped),2)), " years old.","\n\n" ] )

    outfile.close()
        

    
    return replacement


# Function to read lines from input file, get LDOS info from Cisco REST APIs and save LDOS info into an output file 
def get_ldos(num,infile,output,token,now,LDOS_buffer):

    # 'serial_block' is dictionary to store serial number key to site location value
    serial_block={}

    
    # Read 'num' number of lines from input file.  Save each serial number & site id as a key value pair. 
    # This is used to check if CCW-R output entry matches the serial number
    for r in range(num):
        serial = infile.readline()
        if len(serial.split()) == 2:
            serial_block[serial.split()[0]] = serial.split()[1]
        else:
            serial_block[serial.split()[0]] = "Unknown"
        
    # Creating a string of just the serial numbers.  This is used in the CCW-R API payload
    s_block = str([i for i in serial_block]).strip('[]')

    # Before calling any Cisco API, check if auth token needs to be refreshed.
    token,now = refreshtoken(token,now)

    # Calling CCW-R API for the block of serial numbers and save the output in a dictionary  
    ccw_r_response = ccw_r_api(s_block,token)



    # Iterate through the CCW-R API output
    for dev in ccw_r_response['instances']:

        # Check if the Serial Number field is in each CCW-R output entry.     
        if ("serialNumber" in dev):
            
            # LDOS data for each device will be saved in this dictionary
            device = {}
            
            # Check if the serial number in the CCW-R entry matches any of the serial numbers from the input file
            # If then save the relevant fields to the 'device' dictonary which will be save to output file later.
            if dev["serialNumber"] in serial_block:
                if serial_block[dev["serialNumber"]] == "Unknown":
                    device["Site"] = dev["endCustomer"]["address"]["city"]+','+dev["endCustomer"]["address"]["country"]
                else:
                    device["Site"] = serial_block[dev["serialNumber"]]
                
                # Removing the serial number so if there is duplicate entry in another CCW-R output entry, then it will not match and we don't get a duplicate
                del serial_block[dev["serialNumber"]]
                
                device["Serial"] = dev["serialNumber"]
                device["Product"] = dev["product"]["number"]
                
                # checking if device serial has a LDOS entry.  If so call the Cisco EOX API to get replacement product ID
                if "lastDateOfSupport" in dev:
                    
                    # before API call, let's check if auth token needs to be refreshed
                    token,now = refreshtoken(token,now)
                    
                    # checking if there is already a LDOS product entry in the temporary dictionary.  
                    # If there is then get the LDOS date and replacement from this dictionary so we don't need to call EOX API unnecessarily
                    if device["Product"] in LDOS_buffer:
                        device["LDOSDate"] = LDOS_buffer[device["Product"]]["LDOSDate"]
                        device["Replacement"] =  LDOS_buffer[device["Product"]]["Replacement"]

                    # If the product id is not in the temporary dictionary, then call the EOX API and save the info
                    else:
                        eol_prod = dev["product"]["number"]
                        replacement_is_ldos = True
                        ldos_date =  ""
                        while replacement_is_ldos:
                            eox_api_response = eox_api(eol_prod,token)
                    
                            if (eox_api_response["EOXRecord"][0]["LastDateOfSupport"]["value"] != "") and \
                            (eox_api_response["EOXRecord"][0]["EOXMigrationDetails"]["MigrationProductId"] !=""):
                                eol_prod = eox_api_response["EOXRecord"][0]["EOXMigrationDetails"]["MigrationProductId"]
                                ldos_date = eox_api_response["EOXRecord"][0]["LastDateOfSupport"]["value"]
                            else:
                                replacement_is_ldos = False
                            
                        device["LDOSDate"] = ldos_date
                        if eol_prod.startswith('C9500') and not(eol_prod.endswith('-A') or eol_prod.endswith('-E')):
                            eol_prod = eol_prod + '-A'
                        device["Replacement"] = eol_prod
                        LDOS_buffer[device["Product"]]= {"LDOSDate": device["LDOSDate"],"Replacement":device["Replacement"]}
                        
                # if there is no LDOS date for the product, then just enter blank for LDOS date and replacement        
                else :
                    device["LDOSDate"] = ""
                    device["Replacement"] = ""
                
                # Check if there is a ship date for the device.  This is used to calculate how long the device has been installed.
                if "shipDate" in dev:    
                    device["ShipDate"] = dev["shipDate"]
                else:
                    device["ShipDate"] = ""
                    
                # save the device LDOS info to the output file    
                # outfile.writelines(json.dumps(device)+"\n")


                output.append(device)
                
    return token,now,LDOS_buffer,output
                
# Function to determine how many Cisco API calls are needed.
def lookup_ldos():
    
    # opening the source file that has hte serial numbers and site ids.
    # opening the destinaton file to save the LDOS info
    infile = open(input_file,'r')
    
    output = []

    # Group serial numbers in blocks of 20.  This so we can call CCW-R API 20 serial numbers at a time.
    num_of_lines = len(open(input_file).readlines())
    a,b = divmod(num_of_lines, 20)
    
    # this is used to determine percentage completed for LDOS processing
    i = 0  
    
    # Get a new API token    
    token,now = getnewtoken()
    
    # This is used to remember if we have seen a product ID before.  This will reduce number of calls to EOX API
    LDOS_buffer={}

    # Get LDOS info in blocks of 20 serial numbers  
    for p in range(a):
        token,now,LDOS_buffer,output = get_ldos(20,infile,output,token,now,LDOS_buffer)
        i += 20
        # print out completion progress
        print(divmod(i*100,num_of_lines)[0],"%"," Complete")

    # Get LDOS info for the remainder after block of 20 have been processed
    if b != 0:
        token,now,LDOS_buffer,output = get_ldos(b,infile,output,token,now,LDOS_buffer)
    print("100%"," Complete")
                           
    return output
 





def plot_ldos_by_quantity(inventory,first_year,last_year):

    LDOS_sum = {}
    for year in range(first_year,last_year+1):
        num = 0
        for dev in inventory:
            if str(year) in dev["LDOSDate"]:
                num += 1
        LDOS_sum[str(year)] = num
    
    # x-coordinates of left sides of bars  
    left = [i for i in range(len(LDOS_sum))]

    # heights of bars 
    height = [LDOS_sum[i] for i in LDOS_sum] 

    # labels for bars 
    tick_label = [i for i in LDOS_sum]

    plt.figure(figsize=(10,5))
    
    # plotting a bar chart 
    plt.bar(left, height, tick_label = tick_label, 
            width = 0.8, color = ['red', 'green']) 

    # naming the x-axis 
    plt.xlabel('Year',fontsize=26) 
    # naming the y-axis 
    plt.ylabel('Quantity',fontsize=26) 
    # plot title 
    plt.title('Number of LDOS Gear', fontsize=30)
    plt.show()
    return

def plot_ldos_by_age(inventory,first_year,last_year):

    LDOS_sum = {}
    for year in range(first_year, last_year+1):
        years_shipped = []
        for dev in inventory:
            if str(year) in dev["LDOSDate"]:
                if dev["ShipDate"] != "":
                    yr = int(dev["ShipDate"][0:4])
                    mo= int(dev["ShipDate"][5:7])
                    day= int(dev["ShipDate"][8:10])
                    f_date = date(yr, mo, day)
                    l_date = date(2020, 10, 19)
                    delta = l_date - f_date
                    years_shipped.append(delta.days/365)
        if years_shipped:
            LDOS_sum[str(year)] = round(statistics.median(years_shipped),2)
        else:
            LDOS_sum[str(year)] = 0
    
  
    
    # x-coordinates of left sides of bars  
    left = [i for i in range(len(LDOS_sum))]

    # heights of bars 
    height = [LDOS_sum[i] for i in LDOS_sum] 

    # labels for bars 
    tick_label = [i for i in LDOS_sum]

    plt.figure(figsize=(10,5))
    plt.axhline(y=6)
    
    # plotting a bar chart 
    plt.bar(left, height, tick_label = tick_label, 
            width = 0.8, color = ['red', 'green']) 

    # naming the x-axis 
    plt.xlabel('Year',fontsize=26) 
    # naming the y-axis 
    plt.ylabel('Age (in years)',fontsize=26) 
    # plot title 
    plt.title('Median Age of LDOS Gear', fontsize=30) 
    # function to show the plot 
    plt.show()
    return



def plot_ldos_by_site(inventory,first_year,last_year):

    LDOS_sum = {}
    for year in range(first_year, last_year+1):
        for dev in inventory:
            if str(year) in dev["LDOSDate"]:
                if dev['Site'] in LDOS_sum:
                    if str(year) in LDOS_sum[dev['Site']]:
                        LDOS_sum[dev['Site']][str(year)] += 1
                    else:
                        LDOS_sum[dev['Site']][str(year)] = 1
                else:
                    LDOS_sum[dev['Site']] = {}
                    LDOS_sum[dev['Site']][str(year)] = 1
   

            
    labels = [i for i in LDOS_sum]
    y={}
    for i in LDOS_sum:
        for year in range(first_year, last_year+1):
            if str(year) in LDOS_sum[i]:
                if str(year) in y:
                    y[str(year)].append(LDOS_sum[i][str(year)])
                else:
                    y[str(year)] = []
                    y[str(year)].append(LDOS_sum[i][str(year)])
            elif str(year) in y:
                y[str(year)].append([0])
            else:
                y[str(year)] = [0]
   

    width = 0.35       # the width of the bars: can also be len(x) sequence


    fig, ax = plt.subplots(figsize=(20, 10))
    for year in y:
        ax.barh(labels, y[year], width,  label=year)


    
    ax.set_xlabel('Quantity',fontsize=26)
    ax.set_title('LDOS Quantity by Site', fontsize=30)
    ax.legend()
    plt.show()

    return


def create_bom(replacement,first_year,last_year):
    from api_cred import c_token
    url = "https://api-test.cisco.com/commerce/EST/v2/POE/sync/createEstimate"
    
    line_num = 1
    p = {"ProcessQuote":{"DataArea":{"Quote":[{"QuoteHeader":{"Extension":[{"ID":[{"value":"","typeCode":"PriceListShortName"},{"value":"Resale","typeCode":"IntendedUseCode"}],"ValueText":[{"value":"LDOS-Test","typeCode":"Estimate Name"},{"value":"N","typeCode":"SweepsIndicator"}]}],"Status":[{"Code":{"value":"VALID","typeCode":"EstimateStatus"}}],"Party":[{"role":"End Customer","Name":[{"value":"ALEXANDER, O"}],"Location":[{"Address":[{"AddressLine":[{"value":"850 BRYANT ST","sequenceNumber":1},{"value":"","sequenceNumber":2},{"value":"","sequenceNumber":3}],"CityName":{"value":"SAN FRANCISCO"},"CountrySubDivisionCode":[{"value":"CA"}],"CountryCode":{"value":"US"},"PostalCode":{"value":"94103-4603"}}]}],"Contact":[{"PersonName":[{"PreferredName":[{"value":"TEST"}]}],"TelephoneCommunication":[{"Extension":[{"ValueText":[{"value":"12345","typeCode":"Contact Fax"}]}],"FormattedNumber":{"value":"987654321","typeCode":"Contact Phone"}}],"EMailAddressCommunication":[{"EMailAddressID":{"value":"asd123@gmail.com"}}]}]},{"role":"Install Site","Name":[{"value":"ONEIDA TRIBE OF WISCONSIN"}],"Location":[{"Address":[{"AddressLine":[{"value":"2170 AIRPORT DRIVE","sequenceNumber":1},{"value":"GAMING ADMINISTRATION","sequenceNumber":2},{"value":"","sequenceNumber":3}],"CityName":{"value":"GREEN BAY"},"CountrySubDivisionCode":[{"value":"WI"}],"CountryCode":{"value":"US"},"PostalCode":{"value":"54313"}}]}],"Contact":[{"PersonName":[{"PreferredName":[{"value":"Indus"}]}],"TelephoneCommunication":[{}],"EMailAddressCommunication":[{"EMailAddressID":{"value":"pqrs@cisco.com"}}]}]}],"BillToParty":{"ID":[{"value":"","typeCode":"BID"}],"Location":[{"Address":[{"AddressLine":[{"value":"8900 CANNOT SHIP ROAD","sequenceNumber":1}],"CityName":{"value":"LOS ANGELES"},"CountrySubDivisionCode":[{"value":"CA"}],"CountryCode":{"value":"US"},"PostalCode":{"value":"90017"}}]}]},"EffectiveTimePeriod":{"EndDateTime":"2013-08-04T07:00:00Z"},"QualificationTerm":[{}]},"QuoteLine":[\
        ]}]},"ApplicationArea":{"Sender":{"LogicalID":{"value":"847291911"},"ComponentID":{"value":"B2B-3.0"},"ReferenceID":{"value":"Tech Data"}},"Receiver":[{"LogicalID":{"value":"364132837"},"ID":[{"value":"Cisco Systems Inc."}]}],"CreationDateTime":"2017-01-07T21:00:59","BODID":{"value":"10-2056548"}}}}

    
    for year in range(first_year, last_year+1):
        for sku in replacement[str(year)]:
            if sku != "":
                line =  {"LineNumberID":{"value":""},"Status":[{"Reason":[{"value":"VALID"}]}],"Item":{"Extension":[{"Quantity":[{"value":0,"unitCode":"each"}]}],"ID":{"value":"","schemeAgencyID":"Cisco","typeCode":"PartNumber"},"Description":[{"value":"test"}],"Classification":[{"Extension":[{"ValueText":[{"value":"CONFIGURABLE","typeCode":"ItemType"}]}],"UNSPSCCode":{"value":"43222609"}}],"Specification":[{"Property":[{"Extension":[{"ValueText":[{"value":"1900-ISR-LOW-MODEL:PRODUCTNAME|CISCO1921-SEC/K9","typeCode":"ConfigurationPath"},{"value":"C1570456","typeCode":"ProductConfigurationReference"},{"value":"USER","typeCode":"ConfigurationSelectCode"},{"value":"N","typeCode":"BundleIndicator"},{"value":"INVALID","typeCode":"VerifiedConfigurationIndicator"},{"value":"2015-07-07T18:41:32Z","typeCode":"ValidationDateTime"}]}],"ParentID":{"value":"0"},"NameValue":{"value":"1.0","name":"CCWLineNumber"},"EffectiveTimePeriod":{"Duration":"P0Y0M35DT0H0M"}}]}]}}            
                line["LineNumberID"]["value"] = str(line_num)
                line_num += 1
                line["Item"]["Extension"][0]["Quantity"][0]["value"] = replacement[str(year)][sku]
                line["Item"]["ID"]["value"] = sku
                p["ProcessQuote"]["DataArea"]["Quote"][0]["QuoteLine"].append(line)

    if len(p["ProcessQuote"]["DataArea"]["Quote"][0]["QuoteLine"]) == 0:
        print("No BOM is built")
        return            
    
    payload=json.dumps(p)
    
    ctoken = 'Bearer ' + c_token
    headers = {
      'Accept': 'application/json',
      'Authorization': ctoken,
      'Content-Type': 'application/json'
    }

    response = requests.request("POST", url, headers=headers, data = payload)
    print(response.text.encode('utf8'))

    eresponse = json.loads(response.text)
    estimate = eresponse["AcknowledgeQuote"]['DataArea']['Quote'][0]['QuoteHeader']['ID']['value']

    print("New CCW Estimate created: ",estimate)
    return 


def main():
    inventory = lookup_ldos()
                
    replacement=show_ldos(inventory, 2020,2025)
    create_bom(replacement,2020,2025)
    # Generate graphs to show LDOS quantity, median age and locations from 2020 to 2025
    plot_ldos_by_quantity(inventory,2020,2025)
    plot_ldos_by_age(inventory,2020,2025)
    plot_ldos_by_site(inventory,2020,2025)



if __name__ == "__main__":
#    main()
    inventory = lookup_ldos()
                
    replacement=show_ldos(inventory, 2020,2025)


In [ ]:
plot_ldos_by_quantity(inventory,2018,2025)
    plot_ldos_by_age(inventory,2018,2025)
    plot_ldos_by_site(inventory,2018,2022)
